In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.modules.loss import _Loss 
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [4]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

/home/muahmmad/.local/lib/python3.10/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


In [5]:
# Initialize generator 
generator = Generator().cuda()
generator.load_state_dict(torch.load("./saved_models/G/generator_795.pth"))

/tmp/ipykernel_89654/2724864579.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load("./saved_models/G/generator_795.pth"))


<All keys matched successfully>

In [6]:
generator.eval()

Generator(
  (linear_encoding): Linear(in_features=384, out_features=512, bias=True)
  (position_encoding): LearnedPositionalEncoding()
  (pe_dropout): Dropout(p=0.0, inplace=False)
  (transformer): TransformerModel(
    (net): IntermediateSequential(
      (0): Residual(
        (fn): PreNormDrop(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (fn): SelfAttention(
            (qkv): Linear(in_features=512, out_features=1536, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=512, out_features=512, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (1): Residual(
        (fn): PreNorm(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=512, out_features=256, bias=True)
           

In [7]:
path='/home/muahmmad/projects/Image_enhancement/Enhancement_Dataset'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
i=1
for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx)
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx).cuda()
    #print(imgx.shape)
    output=generator(imgx)
    out=output[3].data
    save_image(out, "./test/output/"+item, nrow=5, normalize=True)
    i=i+1


ValueError: invalid literal for int() with base 10: '9852_no_fish_f000020'

In [8]:
def compute_psnr(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
def compute_mse(img1,img2):
    mse=np.mean( (img1/255. - img2/255.) ** 2 )
    return mse

In [9]:
path1='./test/GT/'#要改
path2='./test/output/'#要改
path_list = os.listdir(path1)
path_list.sort(key=lambda x:int(x.split('.')[0]))
PSNR=[]

for item in path_list:
    impath1=path1+item
    impath2=path2+item
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    #print(imgx.shape)
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
    

    psnr=(psnr1+psnr2+psnr3)/3.0

    PSNR.append(psnr)



In [10]:
PSNR=np.array(PSNR)    
print(PSNR.mean())

24.360124160989553
